In [1]:
!pip install groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 8.2 MB/s eta 0:00:00


In [8]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

In [9]:
from groq import Groq
import os

# Initialize client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [10]:
MODEL_CONFIG = {

    "technical": {
        "system_prompt": """You are a Technical Expert.
You provide precise, structured, code-focused solutions.
Explain errors clearly and provide corrected code when possible."""
    },

    "billing": {
        "system_prompt": """You are a Billing Support Expert.
You are empathetic, professional, and help users with payments,
refunds, and subscription issues according to company policies."""
    },

    "general": {
        "system_prompt": """You are a General Customer Support Assistant.
You answer casual questions and help users politely."""
    }

}

In [14]:
def route_prompt(user_input):

    router_prompt = f"""
Classify the following text into one of these categories:
technical
billing
general

Return ONLY the category name.

Text: {user_input}
"""

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",   # ✅ FIXED
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a routing classifier."},
            {"role": "user", "content": router_prompt}
        ]
    )

    return response.choices[0].message.content.strip().lower()

In [15]:
def process_request(user_input):

    category = route_prompt(user_input)

    print("Routed to:", category)

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",   # ✅ FIXED
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content

In [16]:
user_input = "My Python code gives IndexError when accessing list"

result = process_request(user_input)

print("\nFinal Response:\n", result)

Routed to: technical

Final Response:
 # Step-by-step analysis of the problem:
1. **Understanding the Error**: An `IndexError` in Python occurs when you try to access an element in a list using an index that does not exist. This can happen if the index is out of range (either too large or too small) or if the list is empty.
2. **Common Causes**: This error often arises from:
   - Trying to access an element at an index that is greater than or equal to the length of the list.
   - Attempting to access an element in an empty list.
   - Misunderstanding how list indices work in Python (0-based indexing).
3. **Identifying the Issue**: Without seeing your specific code, the issue could be due to incorrect indexing, a logic error in your loop or conditional statements that lead to accessing the list, or misunderstanding the size of your list.

# Fixed solution:
Since you haven't provided your specific code, let's consider a common scenario where this might happen and how to fix it. Suppose y

In [17]:
user_input = "I was charged twice for my subscription"

result = process_request(user_input)

print("\nFinal Response:\n", result)

Routed to: billing

Final Response:
 I'm so sorry to hear that you were charged twice for your subscription. I'm here to help you resolve this issue as quickly as possible.

To better assist you, could you please provide me with some more information about the duplicate charge? This will help me to investigate and correct the issue:

1. Your subscription account name or ID (if you have it handy)
2. The date of the duplicate charge
3. The amount that was charged twice
4. The payment method used (e.g., credit card, PayPal, etc.)

Additionally, I want to let you know that our company policy is to promptly refund any duplicate or incorrect charges. If our investigation confirms that you were indeed charged twice, I will work with our billing team to process a refund for the duplicate charge as soon as possible.

Please know that I'm committed to making things right, and I appreciate your patience and cooperation as we work to resolve this issue. If you have any questions or concerns, feel 

In [18]:
user_input = "Hello, how are you?"

result = process_request(user_input)

print("\nFinal Response:\n", result)

Routed to: general

Final Response:
 Hello. I'm doing well, thank you for asking. How can I assist you today? Is there something I can help you with or would you like to chat?
